# A Evolução do Projeto: Gerador de Mapas de Calor

Este notebook documenta a jornada de desenvolvimento do nosso sistema de análise de fluxo, desde a prova de conceito inicial até a lógica robusta que serve de base para a aplicação final em Gradio. Cada seção representa uma fase de experimentação, com seu código original e uma análise crítica dos resultados, sucessos e fracassos.

## Fase 1: O Rascunho Inicial (gerador_mapas_calor (rascunhos bases))

**Objetivo:** Provar o conceito mais básico: é possível usar YOLO para detectar pessoas e plotar suas posições para criar algum tipo de mapa de calor?

# O que fazer inicialmente?

* Obter um vídeo real com movimentação de pessoas;
* Processar com YOLOv8 + rastreador (DeepSORT);
* Extrair as posições de cada ID em cada frame;  
* Plotar essas posições como mapa de calor;
* Testar em outras situações.

In [1]:
# Instalando as bibliotecas/dependências necessárias:

!pip install ultralytics deep_sort_realtime opencv-python matplotlib

     ---------------------------------------- 8.4/8.4 MB 1.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Upload do vídeo
from google.colab import files
uploaded = files.upload()

# Assumimos que só há um arquivo enviado e pegamos o nome:
video_path = list(uploaded.keys())[0]
print("Vídeo carregado:", video_path)

In [ ]:
# Testando:

import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt

# --- 1. Carregar o modelo YOLOv8 pré-treinado ---
model = YOLO("yolov8n.pt")  # "n" = nano, é leve

# --- 2. Iniciar o DeepSORT para rastrear pessoas ---
tracker = DeepSort(max_age=15)

# --- 3. Carregar o vídeo ---
video_path = "/content/MOT20-01-raw.webm"  # Caminho do vídeo
cap = cv2.VideoCapture(video_path)

# Listas para armazenar posições rastreadas
all_x = []
all_y = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- 4. Rodar YOLOv8 para detectar pessoas ---
    results = model(frame)[0]
    detections = []

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = result
        if int(cls) == 0:  # classe 0 = pessoa
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # --- 5. Atualizar o rastreador com as detecções ---
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()  # left, top, right, bottom
        x1, y1, x2, y2 = ltrb
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        all_x.append(cx)
        all_y.append(cy)

cap.release()

# --- 6. Criar o mapa de calor com os pontos acumulados ---
plt.figure(figsize=(10, 6))
plt.hist2d(all_x, all_y, bins=100, cmap='hot')
plt.colorbar(label="Intensidade de presença")
plt.gca().invert_yaxis()  # Inverter eixo Y para coincidir com imagem
plt.title("Mapa de Calor da Movimentação de Pessoas")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

### Análise da Fase 1

***O que funcionou:**

**- Prova de Conceito:** Sucesso! Conseguimos provar que era possível usar YOLO para detectar pessoas, um rastreador (DeepSORT) para segui-las e Matplotlib para gerar uma visualização baseada nas suas posições. O fluxo básico foi validado.

**- Integração de Bibliotecas:** A integração entre ultralytics, deep_sort_realtime e matplotlib funcionou como esperado.

**O que deu errado / Limitações:**

**- Complexidade do DeepSORT:** A integração com o DeepSORT, embora funcional, exigia uma formatação de dados muito específica (detections.append(...)), tornando o código verboso e propenso a erros. Descobrimos mais tarde que o próprio YOLOv8 tinha um rastreador embutido muito mais simples de usar.

**- Qualidade da Visualização:** O resultado final, usando plt.hist2d, não é um mapa de calor verdadeiro no contexto de visão computacional. É um histograma 2D, que cria uma imagem quadriculada e de baixa resolução. Ele não se sobrepõe ao vídeo e não tem a aparência suave e profissional que buscávamos.

**- Eficiência:** O processo era lento e não havia otimizações como pular frames (FRAME_SKIP).

**Conclusão:** Esta fase foi um sucesso como um primeiro passo, mas o resultado final era tecnicamente pobre e visualmente insatisfatório. Ficou claro que precisávamos de uma técnica de visualização melhor (com OpenCV) e um método de rastreamento mais simples.

## Fase 2: Refinamento e Feedback (map)

**Objetivo:** Abandonar o hist2d em favor de uma sobreposição de mapa de calor real no vídeo. Implementar um rastreador mais simples e começar a extrair métricas quantitativas, incorporando o feedback do professor.

In [ ]:
# ==================================================================================================
# FASE 1: GERADOR DE MAPA DE CALOR ESTÁTICO - V1.1
# DESCRIÇÃO: Script otimizado para gerar uma imagem de mapa de calor de alta qualidade
#            e extrair dados quantitativos da análise de um arquivo de vídeo.
# ==================================================================================================

# --------------------------------------------------------------------------------------------------
# 1. INSTALAÇÕES E IMPORTS
# --------------------------------------------------------------------------------------------------
!pip install ultralytics tqdm -q

import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import os
from tqdm import tqdm # Para a barra de progresso

# --------------------------------------------------------------------------------------------------
# 2. CONFIGURAÇÃO CENTRALIZADA
# --------------------------------------------------------------------------------------------------
# Acesso ao Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/visao_computacional/gerador-mapas-calor/Código/'
VIDEO_PATH = os.path.join(BASE_DIR, 'videos/MOT20-01-raw.webm')
RESULTS_PATH = os.path.join(BASE_DIR, 'resultados/')

# --- Configurações do Modelo YOLO ---
# Usando o 'yolov8n.pt' é leve, rápido e mais comum nas aplicações.
MODEL_NAME = 'yolov8n.pt'
CONF_THRESHOLD = 0.4 # Limiar de confiança: Apenas detecções com score > 0.4 são consideradas.

# --- Configurações de Performance ---
# Pula N frames para acelerar. 2 significa que 1 em cada 3 frames é analisado.
FRAME_SKIP = 2

# --- Configurações Visuais do Mapa de Calor ---
# JUSTIFICATIVA: Um kernel grande (95,95) foi escolhido para criar manchas bem difusas,
# ideal para visualizar aglomerações em vídeos de longa distância.
GAUSSIAN_KERNEL_SIZE = (95, 95)
# "Força" de cada ponto. Um valor maior torna as áreas de pico mais "vermelhas".
HEAT_POINT_INTENSITY = 5
# Transparência do mapa sobre o vídeo.
HEATMAP_ALPHA = 0.4
COLOR_MAP = cv2.COLORMAP_JET

# --------------------------------------------------------------------------------------------------
# 3. LÓGICA PRINCIPAL DO SCRIPT
# --------------------------------------------------------------------------------------------------

def generate_static_heatmap():
    """
    Função principal que encapsula a lógica para gerar o mapa de calor estático.
    Ela processa um vídeo, coleta pontos de calor, gera uma imagem de sobreposição
    e imprime uma análise quantitativa básica dos dados.
    """
    os.makedirs(RESULTS_PATH, exist_ok=True)

    # --- Carregamento e Verificação ---
    print(f"Carregando modelo '{MODEL_NAME}'...")
    model = YOLO(MODEL_NAME)

    print(f"Abrindo vídeo: '{VIDEO_PATH}'...")
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("ERRO CRÍTICO: Não foi possível abrir o arquivo de vídeo. Verifique o caminho.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # --- Coleta de Pontos de Calor ---
    heat_points = []
    background_frame_captured = False # Flag para pegar o primeiro frame
    background_frame = None
    frames_processed_count = 0

    with tqdm(total=total_frames, desc="Analisando vídeo") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                pbar.update(total_frames - pbar.n)
                break

            pbar.update(1)

            # MELHORIA: Pega o primeiro frame válido como fundo e evita reabrir o vídeo
            if not background_frame_captured:
                background_frame = frame.copy()
                background_frame_captured = True

            # Pula frames conforme a configuração
            if pbar.n % (FRAME_SKIP + 1) != 0:
                continue

            frames_processed_count += 1
            results = model(frame, classes=[0], conf=CONF_THRESHOLD, verbose=False)

            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    center_x, bottom_y = int((x1 + x2) / 2), int(y2)
                    heat_points.append((center_x, bottom_y))

    cap.release()

    # --- Geração do Mapa de Calor Visual ---
    if not background_frame_captured:
        print("ERRO: Não foi possível ler nenhum frame do vídeo para usar como fundo.")
        return

    print("Gerando visualização do mapa de calor...")
    heatmap_matrix = np.zeros((frame_height, frame_width), dtype=np.float32)
    for x, y in heat_points:
        if 0 <= y < frame_height and 0 <= x < frame_width:
            heatmap_matrix[y, x] += HEAT_POINT_INTENSITY

    heatmap_matrix = cv2.GaussianBlur(heatmap_matrix, GAUSSIAN_KERNEL_SIZE, 0)
    heatmap_normalized = cv2.normalize(heatmap_matrix, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    colored_heatmap = cv2.applyColorMap(heatmap_normalized, COLOR_MAP)
    superimposed_img = cv2.addWeighted(background_frame, 1 - HEATMAP_ALPHA, colored_heatmap, HEATMAP_ALPHA, 0)

    # --- Salvamento e Exibição ---
    output_path = os.path.join(RESULTS_PATH, 'mapa_de_calor_ESTATICO_FINAL.png')
    cv2.imwrite(output_path, superimposed_img)
    print(f"\n✅ Visualização do mapa de calor salva em: {output_path}")

    display_height = 600
    display_width = int(frame_width * (display_height / frame_height))
    display_img = cv2.resize(superimposed_img, (display_width, display_height))
    cv2_imshow(display_img)

    # --- NOVA SEÇÃO: Análise Quantitativa (A cereja do bolo) ---
    print("\n--- Análise Quantitativa do Vídeo ---")
    total_detections = len(heat_points)
    avg_detections_per_frame = total_detections / frames_processed_count if frames_processed_count > 0 else 0
    print(f"Total de frames no vídeo: {total_frames}")
    print(f"Frames efetivamente analisados (considerando FRAME_SKIP={FRAME_SKIP}): {frames_processed_count}")
    print(f"Total de detecções de pessoas: {total_detections}")
    print(f"Média de pessoas detectadas por frame analisado: {avg_detections_per_frame:.2f}")
    print("------------------------------------")


# --------------------------------------------------------------------------------------------------
# 4. PONTO DE ENTRADA DO SCRIPT
# --------------------------------------------------------------------------------------------------
if __name__ == '__main__':
    generate_static_heatmap()

### Análise da Fase 2
Esta fase representa um salto de qualidade significativo. Os códigos foram refinados, organizados em seções e funções, e o resultado visual começou a se parecer com o objetivo final.

**O que funcionou:**

**- Estrutura do Código:** A organização com configuração centralizada, funções e um ponto de entrada if __name__ == '__main__' tornou o código muito mais legível e profissional.

**- Visualização com OpenCV:** A troca do hist2d pelo pipeline np.zeros -> GaussianBlur -> normalize -> applyColorMap -> addWeighted foi a decisão correta. Conseguimos gerar um mapa de calor suave e sobreposto ao vídeo.

**- Introdução ao Rastreamento:** Os scripts seguintes nesta fase (não incluídos aqui, mas presentes no arquivo maps_1.ipynb) introduziram a ideia de rastreamento, mesmo que com um tracker SORT simplificado implementado manualmente. Isso nos forçou a pensar em como gerenciar IDs e trajetórias, um passo crucial.

**- Análise de Dados:** A adição de gráficos com Matplotlib/Seaborn e a análise quantitativa mostraram o potencial do projeto para extrair insights reais, não apenas imagens bonitas.

**O que deu errado / Limitações:**

**- FALHA CONCEITUAL:** Detecção vs. Rastreamento: A primeira versão (V1.1, mostrada acima) ainda usava model(frame), que apenas detecta, não rastreia. Isso levou à percepção crítica de que a nossa "Análise Quantitativa" estava fundamentalmente errada, pois contava múltiplas detecções da mesma pessoa como eventos separados. Este foi o aprendizado mais importante da fase.

**- Tracker Manual:** O tracker SORT que implementamos manualmente nas iterações seguintes era muito simplista. Ele perdia IDs facilmente se as pessoas se moviam rápido ou se sobrepunham, mostrando que precisávamos de uma solução mais robusta.

**- Mapa de Calor de "Presença", não de "Fluxo":** Nossa técnica de heatmap, embora melhor, ainda era baseada em pontos de presença. Isso criava "bolhas" de calor onde as pessoas paravam, em vez de mostrar os "caminhos" do movimento. O feedback do professor e a análise das imagens de referência (MOT20-02) mostraram que precisávamos visualizar o fluxo.

**Conclusão:** Esta fase foi essencial para refinar a técnica e, mais importante, para entender as limitações da nossa abordagem. Percebemos que precisávamos de um rastreador de verdade (como o embutido no YOLOv8) e de uma nova lógica de heatmap que visualizasse as trajetórias, e não os pontos.



## Fase 3: A Lógica Final (gerador-mapas-calor - Códigos Finais)

**Objetivo:** Sintetizar todos os aprendizados anteriores em um único notebook coeso, que implementa a lógica final e correta. Este código se tornaria a base exata para os módulos .py da aplicação Gradio.

In [ ]:
# ==================================================================================================
# PROJETO: GERADOR DE MAPAS DE CALOR PARA OTIMIZAÇÃO DE ESPAÇOS PÚBLICOS
# MÓDULO: NOTEBOOK DE EXPERIMENTAÇÃO FINAL (ADAPTATIVO E COM MÚLTIPLAS SAÍDAS)
# VERSÃO: 13.0 - SÍNTESE FINAL, COM FOCO NA QUALIDADE VISUAL DE FLUXO
#
# DESCRIÇÃO: Esta versão final implementa a geração de múltiplos artefatos (vídeo de
#            rastreamento e mapa de calor de fluxo) em uma única passagem. Crucialmente,
#            utiliza a lógica correta de heatmap baseada no DESENHO DAS TRAJETÓRIAS com
#            parâmetros visuais ADAPTATIVOS à escala da cena.
# ==================================================================================================

# --------------------------------------------------------------------------------------------------
# 1. INSTALAÇÕES E IMPORTAÇÕES
# --------------------------------------------------------------------------------------------------
!pip install ultralytics matplotlib tqdm -q

import cv2
import numpy as np
from ultralytics import YOLO
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from collections import defaultdict
import random

print("✅ Bibliotecas e dependências importadas.")

# --------------------------------------------------------------------------------------------------
# 2. BLOCO DE CONFIGURAÇÃO CENTRALIZADA
# --------------------------------------------------------------------------------------------------
# --- Definição de Caminhos (Paths) ---
BASE_DIR = '..'
VIDEO_FILENAME = 'pessoas.mp4' # Teste com 'terminal.mp4' ou 'praca_europa.mp4'
VIDEO_PATH = os.path.join(BASE_DIR, 'data', 'videos_publicos', VIDEO_FILENAME)
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
os.makedirs(os.path.join(OUTPUT_DIR, 'heatmaps'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'tracked_videos'), exist_ok=True)

# --- Configurações do Modelo e Rastreamento ---
MODEL_NAME = 'yolov8s.pt'
CONF_THRESHOLD = 0.3
TRACKING_CLASSES = [0]
FRAME_SKIP = 0

# --- Parâmetros de Visualização (ADAPTATIVOS) ---
# Fatores que serão multiplicados pela altura média das detecções.
ADAPTIVE_LINE_FACTOR = 0.05 # A espessura da linha será 5% da altura média.
ADAPTIVE_BLUR_FACTOR = 1.0  # O raio do blur será 100% da altura média.
HEATMAP_ALPHA = 0.5
COLOR_MAP = cv2.COLORMAP_JET

print("✅ Parâmetros de configuração inicializados.")

# --------------------------------------------------------------------------------------------------
# 3. FUNÇÕES MODULARIZADAS
# --------------------------------------------------------------------------------------------------

def draw_tracking_annotations(frame, boxes_xyxy, track_ids, track_colors):
    """Desenha as anotações de rastreamento (caixas e IDs) de forma customizada."""
    for box, track_id in zip(boxes_xyxy, track_ids):
        x1, y1, x2, y2 = map(int, box)
        if track_id not in track_colors:
            track_colors[track_id] = (random.randint(30, 255), random.randint(30, 255), random.randint(30, 255))
        color = track_colors[track_id]
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID:{track_id}"
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - h - 10), (x1 + w + 5, y1), color, -1)
        cv2.putText(frame, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, lineType=cv2.LINE_AA)
    return frame

def process_video_single_pass(video_path, model, output_video_path, conf_threshold):
    """
    Processa o vídeo em uma única passagem, gerando o vídeo de rastreamento
    e coletando os dados para o mapa de calor (trajetórias e alturas).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): raise IOError(f"ERRO: Falha ao abrir o vídeo: {video_path}")

    w, h, fps = (int(cap.get(p)) for p in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    out_video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    # AQUI ESTÁ A LÓGICA CORRETA: Coletamos a TRAJETÓRIA completa.
    track_history = defaultdict(list)
    detection_heights = []
    track_colors = {}
    first_frame = None
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    with tqdm(total=total_frames, desc="Processando Vídeo (Passagem Única)") as pbar:
        for frame_index in range(total_frames):
            success, frame = cap.read()
            if not success: break
            if first_frame is None: first_frame = frame.copy()

            results = model.track(frame, persist=True, classes=TRACKING_CLASSES, conf=conf_threshold, verbose=False)
            annotated_frame = frame.copy()

            if results[0].boxes.id is not None:
                boxes_xyxy = results[0].boxes.xyxy.cpu().numpy()
                boxes_xywh = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()

                annotated_frame = draw_tracking_annotations(annotated_frame, boxes_xyxy, track_ids, track_colors)
                
                for box, track_id in zip(boxes_xywh, track_ids):
                    center_point = (int(box[0]), int(box[1] + box[3] / 2))
                    track_history[track_id].append(center_point)
                    detection_heights.append(box[3])
            
            out_video.write(annotated_frame)
            pbar.update(1)

    cap.release()
    out_video.release()
    
    avg_height = np.mean(detection_heights) if detection_heights else 30.0
    
    print("✅ Processamento de passagem única concluído.")
    return track_history, first_frame, avg_height

def generate_adaptive_flow_heatmap(background_frame, track_history, avg_height, line_factor, blur_factor):
    """
    Gera o mapa de calor de DENSIDADE DE FLUXO desenhando as trajetórias e aplicando um blur adaptativo.
    """
    if background_frame is None: raise ValueError("Frame de fundo é nulo.")

    h, w, _ = background_frame.shape
    
    # --- CÁLCULO DOS PARÂMETROS ADAPTATIVOS ---
    line_thickness = max(1, int(avg_height * line_factor))
    kernel_size = max(1, int(avg_height * blur_factor))
    if kernel_size % 2 == 0: kernel_size += 1
    gaussian_kernel = (kernel_size, kernel_size)
    
    print(f"\nAltura Média da Detecção: {avg_height:.2f}px.")
    print(f"-> Espessura da Linha Adaptativa: {line_thickness}px")
    print(f"-> Kernel de Blur Adaptativo: {gaussian_kernel}")

    # A LÓGICA CORRETA: Desenha as LINHAS da trajetória em um canvas.
    trajectory_canvas = np.zeros((h, w), dtype=np.float32)
    for path in track_history.values():
        for i in range(len(path) - 1):
            cv2.line(trajectory_canvas, path[i], path[i+1], 1.0, line_thickness)
    
    # Aplica o blur sobre as LINHAS para criar o efeito de "calor" de fluxo.
    trajectory_canvas = cv2.GaussianBlur(trajectory_canvas, gaussian_kernel, 0)
    
    heatmap_norm = cv2.normalize(trajectory_canvas, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    heatmap_color = cv2.applyColorMap(heatmap_norm, COLOR_MAP)

    blended_image = cv2.addWeighted(background_frame, 1 - HEATMAP_ALPHA, heatmap_color, HEATMAP_ALPHA, 0)
    print("✅ Mapa de calor de fluxo adaptativo gerado.")
    return blended_image

# --------------------------------------------------------------------------------------------------
# 4. ORQUESTRAÇÃO E EXECUÇÃO DO EXPERIMENTO
# --------------------------------------------------------------------------------------------------

def run_full_analysis_pipeline():
    """Função principal que orquestra todo o processo."""
    base_filename = os.path.splitext(VIDEO_FILENAME)[0]
    output_heatmap_path = os.path.join(OUTPUT_DIR, 'heatmaps', f"{base_filename}_adaptive_flow_heatmap.png")
    output_video_path = os.path.join(OUTPUT_DIR, 'tracked_videos', f"{base_filename}_tracked.mp4")

    model = YOLO(MODEL_NAME)

    # Coleta as TRAJETÓRIAS completas.
    track_history, first_frame, avg_height = process_video_single_pass(
        VIDEO_PATH, model, output_video_path, CONF_THRESHOLD
    )
    print(f"\n-> Vídeo de rastreamento salvo em: '{output_video_path}'")

    # Usa a função correta para gerar o heatmap de FLUXO.
    final_heatmap_image = generate_adaptive_flow_heatmap(
        first_frame, track_history, avg_height, ADAPTIVE_LINE_FACTOR, ADAPTIVE_BLUR_FACTOR
    )
    
    cv2.imwrite(output_heatmap_path, final_heatmap_image)
    print(f"-> Imagem do mapa de calor salva em: '{output_heatmap_path}'")

    heatmap_rgb = cv2.cvtColor(final_heatmap_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(16, 9))
    plt.imshow(heatmap_rgb)
    plt.title(f"Mapa de Calor de Fluxo Adaptativo - {VIDEO_FILENAME}", fontsize=16)
    plt.axis('off')
    plt.show()

if __name__ == '__main__':
    run_full_analysis_pipeline()
    print("\n--- FIM DA ETAPA DE EXPERIMENTAÇÃO. TODOS OS ARTEFATOS FORAM GERADOS. ---")

### Análise da Fase 3 (e Versões Subsequentes)

**O que funcionou (As Decisões Finais):**

**- YOLOv8 Tracker Embutido:** A decisão de abandonar trackers externos (DeepSORT) ou manuais (SORT-like) em favor do model.track() do YOLOv8 foi a mais acertada. Simplificou o código drasticamente e se mostrou muito mais robusto e preciso.

**- Lógica de Mapa de Calor de Fluxo:** A técnica de desenhar as linhas da trajetória em um canvas e depois aplicar um GaussianBlur foi a única que produziu o resultado visual desejado, replicando a aparência dos mapas de calor profissionais (MOT20-02).

**- Parâmetros Adaptativos:** A introdução de parâmetros adaptativos (como ADAPTIVE_BLUR_FACTOR), que calculam o tamanho do blur e da linha com base na altura média das detecções, foi a chave para tornar a solução robusta e funcional para vídeos com diferentes escalas (câmeras de perto vs. de longe).

**- Estrutura Modular:** Organizar o código em funções claras (process_video_..., generate_heatmap_..., run_experiment) tornou a lógica fácil de entender e, crucialmente, tornou a Etapa 2 (Refatoração) um processo trivial de copiar e colar para os arquivos .py.

**O que deu errado (Bugs e Iterações durante o desenvolvimento):**

**- Erros de Sintaxe:** Durante o desenvolvimento (como visto nas versões 8.x, 9.x), ocorreram bugs simples, mas bloqueantes, como is_opened vs isOpened ou NameError por esquecer de passar um parâmetro. Isso reforçou a importância de testar cada pequena mudança.

**- Geração de Vídeo:** A geração do vídeo de rastreamento para a aplicação Gradio se mostrou um grande desafio. Problemas com codecs (mp4v vs XVID vs avc1) e alertas de antivírus no Windows (que bloqueavam a escrita de arquivos de vídeo) nos forçaram a iterar várias vezes até chegar à solução mais compatível.

**- Qualidade Visual do Heatmap:** As primeiras versões do mapa de calor de fluxo ficavam "desbotadas". A solução foi adicionar um fator de ganho (Gain) para amplificar a intensidade do calor antes da colorização, garantindo cores vivas.

**Conclusão Final:** Esta fase final de experimentação produziu uma lógica de análise completa, robusta e de alta qualidade. Ela resolveu todos os problemas conceituais e técnicos das fases anteriores e nos deu uma base sólida e confiável para construir a aplicação web final.

### Observações:

Esses códigos são um pouco do processo de desenvolvimento (apesar de que hajam mais), fez-se interessante fazer esta linha do tempo para que houve-se um entendimento maior acerca do que estava sendo ou não realizado de forma eficiente.

#### Etapas do Processo em que este notebook faz parte:

✅ Etapa 0: Configuração Completa do Ambiente e Estrutura

Ambiente virtual, dependências, estrutura de pastas e repositório organizados.

✅ Etapa 1: A Fase de Experimentação (O Notebook)

Testes com modelos de detecção, rastreamento e extração de dados em notebooks.

A *Etapa 2* é referente a próxima parte do código em que pode ser localizada na pasta ```app```:

🚧 Etapa 2: Refatorando o Código para Módulos Reutilizáveis

Separação das funções em arquivos .py com foco em reutilização e clareza.